In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import  roc_auc_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
import os
# Import PCA from sklearn.decomposition
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

In [28]:

train_set=pd.read_csv('train_set.csv')
test_set=pd.read_csv('test_set.csv')

In [29]:

# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])

X2     2590
X3     2139
X75     456
X76     444
X77     447
X78     447
dtype: int64
X2     1085
X3      971
X75     186
X76     198
X77     195
X78     195
dtype: int64


In [30]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [31]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [32]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [ ]:
scalar=MinMaxScaler()
X = scalar.fit_transform(X)
X_testdata = scalar.transform(X_testdata)




In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# Train Extra Trees Classifier on the training data
clf = ExtraTreesClassifier(
    n_estimators=1000,    # Number of trees in the forest
    max_depth=5,          # Maximum depth of each tree (adjust as needed)
    min_samples_split=9,  # Minimum number of samples required to split a node
    min_samples_leaf=50,  # Minimum number of samples required at each leaf node
    random_state=42
)
clf.fit(X, y)

# Option 1: Using mean threshold
# selector_mean = SelectFromModel(clf, threshold="mean", prefit=True)
# X_selected_mean = selector_mean.transform(X)
# X_test_selected_mean = selector_mean.transform(X_testdata)

# Option 2: Using a custom threshold value
custom_threshold = 0.1  # Set a custom threshold for feature importance
selector_custom = SelectFromModel(clf, threshold=custom_threshold, prefit=True)
X_selected_custom = selector_custom.transform(X)
X_test_selected_custom = selector_custom.transform(X_testdata)

# Option 3: Using a quantile threshold (e.g., top 25% features)
# import numpy as np

# feature_importances = clf.feature_importances_
# quantile_threshold = np.percentile(feature_importances, 90)  # Top 10% most important features
# selector_quantile = SelectFromModel(clf, threshold=quantile_threshold, prefit=True)
# X_selected_quantile = selector_quantile.transform(X)
# X_test_selected_quantile = selector_quantile.transform(X_testdata)

# Get the selected feature names for each threshold
# selected_features_mean = X.columns[selector_mean.get_support()]
# selected_features_custom = X.columns[selector_custom.get_support()]
# selected_features_quantile = X.columns[selector_quantile.get_support()]

# print("Selected features with mean threshold:", selected_features_mean)
# print("Selected features with custom threshold:", selected_features_custom)
# print("Selected features with quantile threshold:", selected_features_quantile)


In [ ]:
import pandas as pd

# Assuming feature names are known
feature_names = [f'X{i}' for i in range(X.shape[1])]  # Replace with actual feature names
X_df = pd.DataFrame(X, columns=feature_names)

# # Then use X_df in the code
# selected_features_mean = X_df.columns[selector_mean.get_support()]
selected_features_custom = X_df.columns[selector_custom.get_support()]
# selected_features_quantile = X_df.columns[selector_quantile.get_support()]

# print("Selected features with mean threshold:", selected_features_mean)
print("Selected features with custom threshold:", selected_features_custom)
# print("Selected features with quantile threshold:", selected_features_quantile)


In [ ]:
import pandas as pd

# Assuming X_testdata is a NumPy array, convert it to a DataFrame first
X_testdata_df = pd.DataFrame(X_testdata, columns=feature_names)  # Use the same feature names as for X_df

# Now you can use selected_features_mean to select columns
X_selected_mean_df = X_testdata_df[selected_features_custom]
X_test_selected_mean_df = pd.DataFrame(X, columns=feature_names)[selected_features_custom]  # Also convert X to DataFrame if needed


X=X_test_selected_mean_df
X_testdata=X_selected_mean_df

X.columns

In [ ]:
# # Get the selected feature indices
# selected_feature_indices = selector.get_support(indices=True)

# # Instead of using X.columns, use the original DataFrame's columns (train_set)
# # Assuming 'Y' and potentially 'RecordId' were dropped from X
# original_features = [col for col in train_set.columns if col not in ['Y', 'RecordId']]

# # Get the names of the selected features using the indices and original features
# selected_features = [original_features[i] for i in selected_feature_indices]

# # Now you can use selected_features to create DataFrames from your arrays
# X_selected_df = pd.DataFrame(X_selected, columns=selected_features, index=train_set.index)
# X_test_selected_df = pd.DataFrame(X_test_selected, columns=selected_features, index=test_set.index)

# print("Selected features:", selected_features)

Selected features: ['X6', 'X17', 'X18', 'X20', 'X22', 'X25', 'X26', 'X28', 'X30', 'X32', 'X34', 'X36', 'X40', 'X42', 'X44', 'X45', 'X46', 'X62', 'X64', 'X69', 'X70']


In [ ]:
# X=X_selected_df
# X_testdata=X_test_selected_df

# X.columns

Index(['X6', 'X17', 'X18', 'X20', 'X22', 'X25', 'X26', 'X28', 'X30', 'X32',
       'X34', 'X36', 'X40', 'X42', 'X44', 'X45', 'X46', 'X62', 'X64', 'X69',
       'X70'],
      dtype='object')

In [ ]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.4, random_state=2)

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
# Initialize the ExtraTreesClassifier with appropriate hyperparameters
clf = ExtraTreesClassifier(
    n_estimators=1000,  # Number of trees in the forest
    max_depth=100,      # Maximum depth of each tree (adjust as needed)
    min_samples_split=9,  # Minimum number of samples required to split a node
    min_samples_leaf=50,   # Minimum number of samples required at each leaf node
    #max_features='sqrt',  # Number of features to consider when looking for the best split
    random_state=42
)
clf.fit(trainX, trainy)

sfs = SequentialFeatureSelector(clf, direction='backward',n_features_to_select=15, scoring='roc_auc', n_jobs=-1)
sfs.fit(X, y)
print(sfs.get_feature_names_out())

In [ ]:
# Initialize the ExtraTreesClassifier with appropriate hyperparameters
clf = ExtraTreesClassifier(
    n_estimators=1000,  # Number of trees in the forest
    max_depth=100,      # Maximum depth of each tree (adjust as needed)
    min_samples_split=9,  # Minimum number of samples required to split a node
    min_samples_leaf=50,   # Minimum number of samples required at each leaf node
    #max_features='sqrt',  # Number of features to consider when looking for the best split
    random_state=42
) random_state=42

clf.fit(trainX, trainy)


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt

y_pred = clf.predict(testX)
y_probs = clf.predict_proba(testX)[:, 1]
fpr, tpr, thresholds = roc_curve(testy, y_probs)



# Calculate AUC
auc_score = roc_auc_score(testy, y_probs)
print(f'AUC: {auc_score}')

In [ ]:
clf.fit(X, y)

ExtraTreesClassifier(max_depth=5, min_samples_leaf=50, min_samples_split=9,
                     n_estimators=1000, random_state=42)

In [ ]:

# Predict probabilities for the test set
y_test_probs = clf.predict_proba(X_testdata)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_test_probs

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)


import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)


[0.00136465 0.01554771 0.00153179 ... 0.00136179 0.00265299 0.00205983]
        RecordId  Y_probability
0         300001       0.001365
1         300002       0.015548
2         300003       0.001532
3         300004       0.001000
4         300005       0.001349
...          ...            ...
105477    405478       0.001019
105478    405479       0.087493
105479    405480       0.001362
105480    405481       0.002653
105481    405482       0.002060

[105482 rows x 2 columns]
